<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395 Redes Neuronales Artificiales 2019-1</h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>
<hr style="height:2px;border:none"/>

## Integrantes

* _**Francisco Olivares - 201473575-8 - fracisco.olivars.14@sansano.usm.cl**_
* _**Gabriel Valenzuela - 201473505-7 - gabriel.valenzuel.14@sansano.usm.cl**_
* _**Felipe Vega - 201473511-1 - felipe.vega.14@sansano.usm.cl**_



<hr style="height:2px;border:none"/>

In [1]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact 

#Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Keras stuff
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

#Que se vea ancho el jupyter
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


#Cantidad de columnas a mostrar con pandas
pd.set_option('display.max_columns',100)

Using TensorFlow backend.


## a) 

In [59]:
# Importando los datos
df = pd.read_csv("data/ozone_data.csv")

print("Registros originales:",df.shape[0])

# Como se quiere predecir el O3 del día siguiente, se crea una nueva columna
from datetime import datetime, timedelta

a_day = timedelta(days=1)
df["is_it_valid"] = False

old_date = False
for index, row in df.iterrows():
    current_date = datetime.strptime(row['registered_on'], "%Y-%m-%d")

    if old_date:
        # Se verifica que haya un día de diferencia
        if old_date == current_date - a_day:
            # Se cambia el campo is_it_valid
            df.at[old_index,'is_it_valid'] = True
            df.at[index,'is_it_valid'] = True
        
    old_date = current_date
    old_index = index
    
df = df[df.is_it_valid == True]
df.drop(columns=['is_it_valid'])
display(df.head())
display(df.describe())

Registros originales: 1629


,registered_on,CO,PM10,PM25,NO,NOX,WD,RH,TEMP,WS,UVA,UVB,O3,is_it_valid
0,2003-12-14,0.172413,0.094241,0.052434,0.016015,0.033293,0.643926,0.670000,0.415679,0.156788,0.035060,0.036334,0.359281,True
1,2003-12-15,0.241378,0.160995,0.119850,0.084537,0.111707,0.618166,0.822219,0.501561,0.155087,0.034838,0.036579,0.359281,True
2,2003-12-16,0.344828,0.227749,0.123596,0.118929,0.174259,0.626491,0.504444,0.694880,0.149334,0.033797,0.038170,0.544910,True
3,2003-12-17,0.275862,0.166230,0.138577,0.103177,0.170368,0.649371,0.503333,0.734857,0.167239,0.027892,0.026847,0.586826,True
4,2003-12-18,0.344828,0.172775,0.183521,0.119716,0.182938,0.624373,0.654444,0.611180,0.175061,0.032705,0.034957,0.610778,True


,CO,PM10,PM25,NO,NOX,WD,RH,TEMP,WS,UVA,UVB,O3
count,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000
mean,0.185925,0.129537,0.119036,0.078211,0.143282,0.688729,0.715991,0.552073,0.157793,0.031703,0.032504,0.415756
std,0.084304,0.064462,0.062839,0.062528,0.085567,0.080498,0.139969,0.132247,0.087221,0.019874,0.007778,0.127267
min,0.000000,0.011780,0.003745,0.000147,0.010739,0.563581,0.155555,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.137931,0.094241,0.082397,0.039823,0.091954,0.643235,0.619166,0.482590,0.128561,0.023959,0.028471,0.335329
50%,0.172413,0.117801,0.112360,0.067997,0.130853,0.663184,0.726667,0.570113,0.143225,0.030216,0.034655,0.413174
75%,0.230017,0.148397,0.142322,0.099501,0.168946,0.698696,0.824444,0.636635,0.159015,0.034672,0.037673,0.496913
max,0.700110,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.117886,0.048340,0.910180


Como el problema a atacar es la predicción de $O_3$ para el día siguiente, en el bloque anterior de código se procedió a cargar los datos y eliminar aquellos registros que no tuvieran un día contiguo.

Después se procede a contabilizar la cantidad de registros por año

In [60]:
register_per_year = {}

for index, row in df.iterrows():
    year = row['registered_on'].split("-")[0]
    if year not in register_per_year:
        register_per_year[year]=0
    register_per_year[year]+=1
    
display(df.describe())
print(register_per_year)

,CO,PM10,PM25,NO,NOX,WD,RH,TEMP,WS,UVA,UVB,O3
count,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000,1610.000000
mean,0.185925,0.129537,0.119036,0.078211,0.143282,0.688729,0.715991,0.552073,0.157793,0.031703,0.032504,0.415756
std,0.084304,0.064462,0.062839,0.062528,0.085567,0.080498,0.139969,0.132247,0.087221,0.019874,0.007778,0.127267
min,0.000000,0.011780,0.003745,0.000147,0.010739,0.563581,0.155555,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.137931,0.094241,0.082397,0.039823,0.091954,0.643235,0.619166,0.482590,0.128561,0.023959,0.028471,0.335329
50%,0.172413,0.117801,0.112360,0.067997,0.130853,0.663184,0.726667,0.570113,0.143225,0.030216,0.034655,0.413174
75%,0.230017,0.148397,0.142322,0.099501,0.168946,0.698696,0.824444,0.636635,0.159015,0.034672,0.037673,0.496913
max,0.700110,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.117886,0.048340,0.910180


{'2003': 18, '2004': 123, '2005': 148, '2006': 149, '2007': 133, '2008': 146, '2009': 120, '2010': 138, '2011': 142, '2012': 149, '2013': 133, '2014': 129, '2015': 40, '2016': 42}


Se aprecia que el dataset tiene 1610 registros, entonces se decide ocupar alrededor del 70% como conjunto de entrenamiento, esto equivalen a 1141 registros, pero como las series de tiempo no son continuas se decide ocupar los primeros años tal que formen alrededor de 1141 registros, entonces por ello se decide utilizar desde el año 2003 al 2011 como entrenamiento

In [32]:
df['registered_year'] = df['registered_on'].apply(lambda x: int(x.split('-')[0]))
df.head()

TRAINING_YEARS = [ 2000 + x for x in range(3, 11 + 1)]

print(TRAINING_YEARS)

df_x_train = df[df.registered_year.isin(TRAINING_YEARS)]
df_x_train.head()

[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011]


,registered_on,CO,PM10,PM25,NO,NOX,WD,RH,TEMP,WS,UVA,UVB,O3,registered_year
0,2003-12-14,0.172413,0.094241,0.052434,0.016015,0.033293,0.643926,0.670000,0.415679,0.156788,0.035060,0.036334,0.359281,2003
1,2003-12-15,0.241378,0.160995,0.119850,0.084537,0.111707,0.618166,0.822219,0.501561,0.155087,0.034838,0.036579,0.359281,2003
2,2003-12-16,0.344828,0.227749,0.123596,0.118929,0.174259,0.626491,0.504444,0.694880,0.149334,0.033797,0.038170,0.544910,2003
3,2003-12-17,0.275862,0.166230,0.138577,0.103177,0.170368,0.649371,0.503333,0.734857,0.167239,0.027892,0.026847,0.586826,2003
4,2003-12-18,0.344828,0.172775,0.183521,0.119716,0.182938,0.624373,0.654444,0.611180,0.175061,0.032705,0.034957,0.610778,2003


In [53]:
from datetime import datetime, timedelta

a_day = timedelta(days=1)
df["future_O3"] = np.nan

old_date = False
for index, row in df.iterrows():
    current_date = datetime.strptime(row['registered_on'], "%Y-%m-%d")

    if old_date:
        # Verifico que efectovamente haya un día de diferencia
        if old_date == current_date - a_day:
            # Procedo a rellenar el campo 'future_O3'
            df.at[old_index,'future_O3'] = row['O3']
        
    old_date = current_date
    old_index = index
    
df = df.dropna()
df.describe()



,CO,PM10,PM25,NO,NOX,WD,RH,TEMP,WS,UVA,UVB,O3,registered_year,future_O3
count,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000
mean,0.186630,0.128245,0.118477,0.076843,0.141060,0.687922,0.718524,0.555625,0.150219,0.032014,0.032754,0.419807,2008.975309,0.420258
std,0.082041,0.059726,0.062762,0.056198,0.079713,0.078605,0.137809,0.129024,0.060307,0.019048,0.007473,0.125342,3.262599,0.126668
min,0.000000,0.011780,0.003745,0.001050,0.010739,0.563581,0.227777,0.000000,0.054738,0.000000,0.000000,0.000000,2003.000000,0.000000
25%,0.137931,0.094241,0.082397,0.039118,0.092092,0.643389,0.623333,0.487274,0.128856,0.024823,0.028741,0.338923,2006.000000,0.340419
50%,0.172413,0.117801,0.108614,0.067148,0.130842,0.662885,0.727222,0.571519,0.143346,0.030389,0.034833,0.413174,2009.000000,0.415324
75%,0.226482,0.147906,0.142322,0.099501,0.166776,0.697851,0.826388,0.637962,0.158605,0.034609,0.037675,0.497006,2012.000000,0.497006
max,0.700110,0.818063,1.000000,0.699134,0.804863,1.000000,1.000000,1.000000,1.000000,0.110983,0.048340,0.910180,2016.000000,0.910180



<hr style="height:2px;border:none"/>